In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("lab09.ipynb")

ModuleNotFoundError: No module named 'otter'

# Lab 9: Pre-Midterm Review

In this **optional lab**, we will review several concepts from earlier in the semester:
1. SQL
1. Pandas
1. Regression/OLS

### Collaboration Policy

Data science is a collaborative activity. While you may talk with others about the labs, we ask that you **write your solutions individually**. If you do discuss the assignments with others, please **include their names** at the top of this notebook.

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn
import math
import matplotlib.pyplot as plt
import lab09_utils
from pathlib import Path
import sqlalchemy
np.random.seed(42)
plt.style.use('fivethirtyeight')
sns.set()
sns.set_context("talk")
%matplotlib inline
!python3 lab09_utils.py

# Part 1: SQL

## Loading the data

For this problem, we'll look at some tropical storm data. The cells below will import the data from a provided CSV and convert it into a SQL server. The data comes from [this Wikipedia article](https://en.wikipedia.org/wiki/Tropical_cyclones_by_year).

In [4]:
db_path = Path('./data/sql/trop_storms.db')

In [5]:
engine = sqlalchemy.create_engine(f"sqlite:///{db_path}")
connection = engine.connect()
inspector = sqlalchemy.inspect(engine)

In [6]:
inspector.get_table_names()

['trop_storm']

In the cell below, we've displayed the first 5 columns of this SQL table for reference.

In [7]:
query = """
SELECT *
FROM trop_storm
GROUP BY Basin
LIMIT 5
"""
pd.read_sql(query, engine).

SyntaxError: invalid syntax (<ipython-input-7-6d2f4d9a0cb4>, line 7)

## Question 1

### Part 1

We want to find how many storms have happened in each region in the past. Write a query to do this, and store the total number of tropical storms in each region in a column called `Tot_Tropical`.

<!--
BEGIN QUESTION
name: q1a
-->

In [8]:
query = """
SELECT Basin, SUM(Num_Tropical) AS Tot_Tropical
FROM trop_storm
GROUP BY Basin
ORDER BY Tot_Tropical DESC
"""


res_q1a = pd.read_sql(query, engine)
res_q1a

,Basin,Tot_Tropical
0,Worldwide,6585
1,Western Pacific,4699
2,Atlantic,2516
3,Eastern Pacific,1591
4,North Indian,1570
5,South Pacific,1301
6,Australia,808
7,South-West Indian,761
8,South Atlantic,10
9,Mediterranean tropical-like cyclone,1


In [9]:
grader.check("q1a")

q1a results: All test cases passed!

### Part 2

While this seems like some useful data, there are some problems. Look at [this Wikipedia article](https://en.wikipedia.org/wiki/Tropical_cyclones_by_year). We can see that for some years, only some regions have data reported. 

Starting at 1968, we can see that data is being reported for all the current regions. Create a query that outputs a result with the same structure as the previous one, but this time, only select data from after the `Year` 1968. Additionally, don't include the row where the `Basin` is Worldwide.

**Hint:** Consider using a subquery.

<!--
BEGIN QUESTION
name: q1b
-->

In [10]:
query = """
SELECT Basin, SUM(Num_Tropical) AS Tot_Tropical
FROM (SELECT * FROM trop_storm WHERE Year >= 1968 AND Basin !='Worldwide')
GROUP BY Basin
ORDER BY Tot_Tropical DESC;
"""

res_q1b = pd.read_sql(query, engine)
res_q1b

,Basin,Tot_Tropical
0,Western Pacific,2199
1,Eastern Pacific,1111
2,Atlantic,892
3,Australia,808
4,South-West Indian,662
5,South Pacific,640
6,North Indian,546
7,South Atlantic,10
8,Mediterranean tropical-like cyclone,1


In [11]:
grader.check("q1b")

q1b results: All test cases passed!

## Question 2

### Part 1

We can see in the data that these tropical storms cause loss of life and other damages. While this is a very inexact method of calculating the damages, let's use the average number of deaths per storm as a proxy for how dangerous these storms are.

Write a query that calculates the average number of deaths per storm in each of the basins. Let the column with this average be called `Deaths per storm`. Make sure that the rows are sorted by the basin name.

<!--
BEGIN QUESTION
name: q2a
-->

In [12]:
tmp = pd.read_sql("""SELECT * FROM trop_storm WHERE Basin = 'Atlantic' AND Year >= 1968""", engine)
tmp['Deaths'].

SyntaxError: invalid syntax (<ipython-input-12-9443c11fc03a>, line 2)

In [13]:
query = """
SELECT Basin,SUM(Deaths)/SUM(Num_Tropical) AS [Deaths per storm]
FROM trop_storm
GROUP BY Basin
ORDER BY Basin
"""


res_q2a = pd.read_sql(query, engine)
res_q2a

,Basin,Deaths per storm
0,Atlantic,75
1,Australia,3
2,Eastern Pacific,5
3,Mediterranean tropical-like cyclone,4
4,North Indian,646
5,South Atlantic,14
6,South Pacific,2
7,South-West Indian,8
8,Western Pacific,306
9,Worldwide,187


In [14]:
grader.check("q2a")

q2a results: All test cases passed!

### Part 2

We can see in the output above that the North Indian Ocean Basin seems to a high rate of deaths per storm; in fact, it's twice as high as the next highest death rate. Let's look into that a bit more. In the following cell, write a query that selects all data in every the row from the North Indian Ocean Basin. Sort the data in decreasing order of number of deaths.

<!--
BEGIN QUESTION
name: q2b
-->

In [15]:
query = """
SELECT *
FROM trop_storm
WHERE Basin = 'North Indian'
ORDER BY Deaths DESC
LIMIT 5
"""


res_q2b = pd.read_sql(query, engine)
res_q2b

,Year,Basin,Num_Tropical,Num_Named,Deaths,Damage
0,1970,North Indian,15,7,500805,$86.4 million
1,2008,North Indian,10,4,138927,$14.7 billion
2,1991,North Indian,8,3,138906,$1.5 billion
3,1965,North Indian,14,6,47000,
4,1942,North Indian,14,5,40000,


In [16]:
grader.check("q2b")

q2b results: All test cases passed!

### Part 3

Do some research into the year with the most deaths. What happened this year that caused so many deaths?

<!--
BEGIN QUESTION
name: q2c
-->

_Type your answer here, replacing this text._

### Part 4

What are some problems with using the number of deaths per storm as a proxy for the danger of tropical storms in each region?
<!--
BEGIN QUESTION
name: q2d
-->

_Type your answer here, replacing this text._

# Part 2: Pandas, Feature Engineering, and Linear Regression

## Question 3: Loading the Diabetes Dataset

### Part 1

To begin, let's load the provided diabetes dataset.

The dataset is contained in the file `./diabetes.txt`. Inspect the file, and use `pd.read_csv` to store it as a DataFrame in the variable `diabetes`.

**Hint:** look into the `sep` argument for `read_csv`.
<!--
BEGIN QUESTION
name: q3a
-->

In [20]:
diabetes = pd.read_csv('./data/diabetes.txt',delimiter="\t")
diabetes.head()

,AGE,SEX,BMI,BP,S1,S2,S3,S4,S5,S6,Y
0,59,F,32.1,101.0,157,93.2,38.0,4,4.8598,87,151
1,48,M,21.6,87.0,183,103.2,70.0,3,3.8918,69,75
2,72,F,30.5,93.0,156,93.6,41.0,4,4.6728,85,141
3,24,M,25.3,84.0,198,131.4,40.0,5,4.8903,89,206
4,50,M,23.0,101.0,192,125.4,52.0,4,4.2905,80,135


### Part 2
Now, let's separate the data into the design matrix $X$ and the response vector $y$.

Note that $y$ is the response of interest, a quantitative measure of disease progression one year after baseline.

<!--
BEGIN QUESTION
name: q3b
-->

In [21]:
diabetes.pop?

Signature: diabetes.pop(item)
Docstring:
Return item and drop from frame. Raise KeyError if not found.

Parameters
----------
item : str
    Column label to be popped

Returns
-------
popped : Series

Examples
--------
>>> df = pd.DataFrame([('falcon', 'bird',    389.0),
...                    ('parrot', 'bird',     24.0),
...                    ('lion',   'mammal',   80.5),
...                    ('monkey', 'mammal', np.nan)],
...                   columns=('name', 'class', 'max_speed'))
>>> df
     name   class  max_speed
0  falcon    bird      389.0
1  parrot    bird       24.0
2    lion  mammal       80.5
3  monkey  mammal        NaN

>>> df.pop('class')
0      bird
1      bird
2    mammal
3    mammal
Name: class, dtype: object

>>> df
     name  max_speed
0  falcon      389.0
1  parrot       24.0
2    lion       80.5
3  monkey        NaN
File:      ~/miniconda3/envs/data100/lib/python3.6/site-packages/pandas/core/generic.py
Type:      method


In [22]:
y = diabetes.pop('Y')
X = diabetes
X.head(10)

,AGE,SEX,BMI,BP,S1,S2,S3,S4,S5,S6
0,59,F,32.1,101.0,157,93.2,38.0,4,4.8598,87
1,48,M,21.6,87.0,183,103.2,70.0,3,3.8918,69
2,72,F,30.5,93.0,156,93.6,41.0,4,4.6728,85
3,24,M,25.3,84.0,198,131.4,40.0,5,4.8903,89
4,50,M,23.0,101.0,192,125.4,52.0,4,4.2905,80
5,23,M,22.6,89.0,139,64.8,61.0,F,4.1897,68
6,36,F,22.0,90.0,160,99.6,50.0,3,3.9512,82
7,66,F,26.2,114.0,255,185.0,56.0,4.55,4.2485,92
8,60,F,32.1,83.0,179,119.4,42.0,4,4.4773,94
9,29,M,30.0,85.0,180,93.4,43.0,4,5.3845,88


In [23]:
grader.check("q3b")

q3b results: All test cases passed!

## Question 4: Pandas & Feature Engineering

Looking at our data, a few possible problems appear. We see that the `SEX` column takes on values "F" and "M", which we will need to convert to a numeric value. In addition, we can see that the values in each column have varying ranges and magnitudes. In order to have a similar scale and range for each of the features, let's normalize them.

If we look at row 5 in the dataset, we can also see something peculiar.

In [24]:
X.iloc[5]

AGE        23
SEX         M
BMI      22.6
BP         89
S1        139
S2       64.8
S3         61
S4          F
S5     4.1897
S6         68
Name: 5, dtype: object

While most of the rows have a number in their `S4` column, this one doesn't. There must have been a mistake when processing the data! Lets try to gauge the scope of this error.

### Part 1
Use a Pandas method to find the number of cells affected by this problem and save it in the variable `res_q4a`.
<!--
BEGIN QUESTION
name: q4a
-->

In [25]:
X['S4'].dtype

dtype('O')

In [32]:
import re
re.findall(r'[^0-9]+','F')

['F']

In [35]:
res_q4a = X[X['S4'].str.match(r'[^0-9]+')].shape[0]
res_q4a

28

In [36]:
grader.check("q4a")

q4a results: All test cases passed!

Clearly this is a problem. If we're trying to run a regression on this dataset, we'll need to have numbers in the columns, not letters. Let's think of some ways we could solve this.

### Part 2

Assign `res_q4b` to a Numpy array containing the number of each of the correct options.

We could:
1. Remove rows with F in the `S4` column.
1. Remove the whole `S4` column.
1. Impute the mean over the `S4` column.
1. Do nothing
1. Impute the most common value (mode) over the `S4` column.


<!--
BEGIN QUESTION
name: q4b
-->

In [39]:
res_q4b = [1,3,5]

In [40]:
grader.check("q4b")

q4b results: All test cases passed!

### Part 3

In the markdown cell below, explain the pros and cons of each of the options for 4b.

<!--
BEGIN QUESTION
name: q4c
-->

_Type your answer here, replacing this text._

### Part 4: Fixing the Data

Fortunately, a friend who worked with processing this data happens to know what the probem was. It appears that the value `'2'` was replaced with the value `'F'` in the `S4` column. In the following cell, correct this error for the `X` dataframe.
<!--
BEGIN QUESTION
name: q4d
-->

In [41]:
X['S4'] = X['S4'].replace({'F':'2'})

In [42]:
grader.check("q4d")

q4d results: All test cases passed!

### Part 5
Now, let's write a function to one-hot-encode the `SEX` column. Consider using `pd.get_dummies` or `sklearn.feature_extraction.DictVectorizer`.

**Hint:** Currently, some the numerical data is of type `str`, so you will need to convert the appropriate column(s) to `int` first.
<!--
BEGIN QUESTION
name: q4e
-->

In [45]:
X['S4'] = X['S4'].astype(float)

In [50]:
def one_hot_encode(data):
    """
    Return the one-hot encoded dataframe of our input data.
    
    Parameters
    -----------
    data: a dataframe that may include non-numerical features
    
    Returns
    -----------
    A one-hot encoded dataframe that only contains numeric features
    
    """
    
    return pd.get_dummies(data)
one_hot_X = one_hot_encode(X)
one_hot_X.head()

,AGE,BMI,BP,S1,S2,S3,S4,S5,S6,SEX_F,SEX_M
0,59,32.1,101.0,157,93.2,38.0,4.0,4.8598,87,1,0
1,48,21.6,87.0,183,103.2,70.0,3.0,3.8918,69,0,1
2,72,30.5,93.0,156,93.6,41.0,4.0,4.6728,85,1,0
3,24,25.3,84.0,198,131.4,40.0,5.0,4.8903,89,0,1
4,50,23.0,101.0,192,125.4,52.0,4.0,4.2905,80,0,1


### Part 6
Do we need to drop any of the columns? Think about what this means with respect to a bias column.
```
BEGIN QUESTION:
name: q4f
```

_Type your answer here, replacing this text._

### Part 7
Finally, let's write a function normalize each of the columns of `X` so that they have mean 0 and standard deviation 1. Make sure that your function also normalizes `y`!
```
BEGIN QUESTION:
name: q2g
```

In [52]:
def normalize(X, y):
    """
    Return the normalized dataframe of our input data.
    
    Parameters
    -----------
    data: a dataframe with numerical features.
    
    Returns
    -----------
    A dataframe that contains our input data, normalized so each column has mean 0 and standard deviation 1.
    
    """
    #from sklearn.preprocessing import StandardScaler
    s_X = (X - X.mean())/X.std()
    s_y = (y-y.mean())/y.std()
    return s_X,s_y
normalized_X, normalized_y = normalize(one_hot_X, y)
normalized_X.head()

,AGE,BMI,BP,S1,S2,S3,S4,S5,S6,SEX_F,SEX_M
0,0.799594,1.295620,0.459320,-0.928693,-0.731236,-0.911418,-0.054438,0.418057,-0.370569,1.064282,-1.064282
1,-0.039522,-1.080955,-0.552878,-0.177423,-0.402430,1.562643,-0.829361,-1.434962,-1.936285,-0.937474,0.937474
2,1.791277,0.933475,-0.119079,-0.957588,-0.718084,-0.679475,-0.054438,0.060087,-0.544537,1.064282,-1.064282
3,-1.870322,-0.243495,-0.769778,0.256002,0.524802,-0.756789,0.720486,0.476443,-0.196600,-0.937474,0.937474
4,0.113044,-0.764079,0.459320,0.082632,0.327519,0.170984,-0.054438,-0.671740,-0.979458,-0.937474,0.937474


## Question 5: Linear Regression

Now that our design matrix is of the right form, we can start to make our regressions.

In the cells below, we've added the loss functions, linear model, and function that minimizes the loss from Lab 8.

In [53]:
def linear_model(thetas, X):
    """
    Return the linear combination of thetas and features as defined above.
    
    Parameters
    -----------
    thetas: a 1D vector representing the parameters of our model ([theta1, theta2, ...])
    X: a 2D dataframe of numeric features (may also be a 2D numpy array)
    
    Returns
    -----------
    A 1D vector representing the linear combination of thetas and features as defined above.
    """
    return X.dot(thetas)

In [54]:
from scipy.optimize import minimize

def l1(y, y_hat):
    return np.abs(y - y_hat)

def l2(y, y_hat):
    return (y - y_hat)**2

def minimize_average_loss(loss_function, model, X, y):
    """
    Minimize the average loss calculated from using different theta vectors, and 
    estimate the optimal theta for the model.
    
    Parameters
    -----------
    loss_function: either the squared or absolute loss functions defined above
    model: the model (as defined in Question 1b)
    X: a 2D dataframe (or numpy array) of numeric features (one-hot encoded)
    y: a 1D vector of tip amounts
    
    Returns
    -----------
    The estimate for the optimal theta vector that minimizes our loss
    """
    return minimize(lambda theta: loss_function(model(theta, X), y).mean(), x0=np.random.rand(X.shape[1]))['x']

### Part 1: L2 Regression
Let's find the model that minimizes the L2 loss using the functions provided above.
<!--
BEGIN QUESTION
name: q5a
-->

In [57]:
res_q5a =minimize_average_loss(l2,linear_model,normalized_X,normalized_y)

In [58]:
grader.check("q5a")

q5a results: All test cases passed!

### Part 2: L1 Regression
Now, let's find the model that minimizes the L1 loss using the functions provided above.
<!--
BEGIN QUESTION
name: q5b
-->

In [59]:
res_q5b = minimize_average_loss(l1,linear_model,normalized_X,normalized_y)

In [60]:
grader.check("q5b")

q5b results: All test cases passed!

### Part 2: Conclusion

So far, we've created two different linear models using different loss functions. In the next few weeks we'll see some ways that you can improve and evaluate your model.

# Solutions

Congrats on finishing the lab. This lab will not be graded, but make sure to come ask for help if you have any questions! 

You don't need to submit this lab, so we haven't created a Gradescope assignment. However, the system will still generate the submission zip file, but you may ignore that!

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [61]:
grader.check_all()

q1a results: All test cases passed!

q1b results: All test cases passed!

q2a results: All test cases passed!

q2b results: All test cases passed!

q3b results: All test cases passed!

q4a results: All test cases passed!

q4b results: All test cases passed!

q4d results: All test cases passed!

q5a results: All test cases passed!

q5b results: All test cases passed!

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False)